In [1]:
%load_ext autoreload
%autoreload 2

# https://scipy-cookbook.readthedocs.io/items/bundle_adjustment.html
import urllib
import bz2
import os
import numpy as np

BASE_URL = "http://grail.cs.washington.edu/projects/bal/data/ladybug/"
FILE_NAME = "problem-49-7776-pre.txt.bz2"
URL = BASE_URL + FILE_NAME

if not os.path.isfile(FILE_NAME):
    urllib.request.urlretrieve(URL, FILE_NAME)

In [2]:
def read_bal_data(file_name):
    with bz2.open(file_name, "rt") as file:
        n_cameras, n_points, n_observations = map(
            int, file.readline().split())

        camera_indices = np.zeros(n_observations, dtype=int)
        points_2d = np.zeros((n_cameras, n_points, 1, 2))

        for i in range(n_observations):
            camera_index, point_index, x, y = file.readline().split()
            points_2d[int(camera_index), int(point_index)] = [float(x), float(y)]

        calib = {}
        camera_params = np.empty(n_cameras * 9)
        for i in range(n_cameras * 9):
            camera_params[i] = float(file.readline())
        camera_params = camera_params.reshape((n_cameras, -1))

        points_3d = np.empty(n_points * 3)
        for i in range(n_points * 3):
            points_3d[i] = float(file.readline())
        points_3d = points_3d.reshape((n_points, 1, -1))

    return camera_params, points_3d, points_2d

In [3]:
camera_params, _, points2d = read_bal_data(FILE_NAME)


In [21]:
'''
cam.rvec = camera_params[0:3]
cam.tvec = camera_params[3:6]
if update_intrinsic:
    cam.fx = camera_params[6]
    cam.fy = camera_params[7]
if update_distort:
    cam.distort  = camera_params[8:13]
'''
import cv2
calib = dict()
for cid, params in enumerate(camera_params):
    calib[cid] = {"R": cv2.Rodrigues(params[0:3])[0], "tvec": params[3:6], "intr": np.array([[params[6], 0, 0], [0, params[6], 0], [0, 0, 1]]), "distort":np.array([params[7], params[8], 0, 0, 0], dtype=float)}

In [22]:
from pyba.CameraNetwork import CameraNetwork
camNet = CameraNetwork(points2d=points2d, calib=calib)

In [23]:
np.mean(camNet.reprojection_error())

174.46979936316617

In [19]:
from pyba.pyba import bundle_adjust
bundle_adjust(camNet, update_distort=False, update_intrinsic=False, max_num_images=1000)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.3465e+09                                    2.84e+12    
       1              3         1.1902e+09      4.16e+09       3.76e+03       1.82e+11    
       2              5         5.6601e+08      6.24e+08       4.46e+02       2.73e+11    
       3              6         2.2869e+08      3.37e+08       6.48e+02       2.98e+10    
       4              7         1.1516e+08      1.14e+08       1.07e+03       2.72e+09    
       5              9         9.0944e+07      2.42e+07       1.26e+02       1.67e+09    
       6             10         6.7738e+07      2.32e+07       1.65e+02       5.55e+08    
       7             11         5.2147e+07      1.56e+07       5.55e+02       1.17e+11    
       8             13         4.3212e+07      8.94e+06       1.04e+02       1.21e+10    
       9             14         4.1966e+07      1.25e+06       2.22e+02       2.50e+10    

 active_mask: array([0., 0., 0., ..., 0., 0., 0.])
        cost: 5400756.22168341
         fun: array([ 7.62405411, 39.71863602, -3.58254309, ..., -0.63300202,
        0.14134521,  0.94948387])
        grad: array([ 42257.63266238, -28401.76627337, -10342.1938385 , ...,
           95.37032207,    106.55077162,   -138.70255996])
         jac: <8032x3637 sparse matrix of type '<class 'numpy.float64'>'
	with 128512 stored elements in Compressed Sparse Row format>
     message: 'The maximum number of function evaluations is exceeded.'
        nfev: 100
        njev: 74
  optimality: 83875795274.10686
      status: 0
     success: False
           x: array([-0.02361388,  0.29976126, -0.47144228, ...,  0.57630795,
        0.36929679, -1.41541872])

In [20]:
np.mean(camNet.reprojection_error())

7.108609315654906